In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import seed
from numpy.random import randint

In [2]:
def Inverte(inicial,final, x):
    
    #posicaox = inicial
    #posicaoy = final
    y=[]
    for j in range(len(x)):
        if j>=inicial and j <= final:
            y.append(int(x[j]))

    #print("y sem reverso",y)
    y.reverse()
    #print("y com reverso",y)

    h=[]
    for i in range(len(x)):
        if i < inicial:    
            h.append(int(x[i]))

        elif i > final:
            h.append(int(x[i]))

        elif i == inicial:
            for w in range(len(y)):
                h.append(int(y[w]))
       
        
    return h        

In [ ]:
def testa_vizinhancas():
    x = np.array([0,1,2,3,4,5,6,7,8,9])
    for i in range (10):
            j = i+1
            while j < 10:

                if i == j:
                    continue

                xBarra = x.copy()
                xBarra = Inverte(i,j,x)
                print("X_",i,"_",j,"_",xBarra)
                j = j+1         
testa_vizinhancas()

In [4]:
def leArquivo(caminho):
    #inicializando as variaveis
    rows = 0
    cols = 0
    #lendo os dados
    f = open(caminho,"r")

    #lendo a primeira linha
    count = 0
    A = []
    RHS = []
    utilidade = []
    upreenchida = False
    
    for linha in f:
        
        if linha =="\n":
            count = count + 1
            continue

        if count ==0:
            rows = int(linha)

        if count == 1:
            cols = int(linha)
            
        
        if count == 2:
            if upreenchida == False:
                for val in linha.split():
                    utilidade.append(float(val))
                upreenchida = True     
                    
            else:
                vetor =[]
                for val in linha.split():
                    vetor.append(float(val))
                A.append(vetor)

        if count == 3:
            for val in linha.split():
                RHS.append(float(val))

    f.close()
    return np.array(A),np.array(RHS),rows,cols,np.array(utilidade)


In [5]:
#historicoMelhor=[0]
#historicoTodos =[0]

def RetornarValorDaFuncaoObj(x, u):
    total=0
    for j in range(col):
        total= total + (x[j]*u[j])
    return total 

def EViavel(a, x, b):
    eviavel= True 
    for i in range(row):
        total=0
        for j in range(col):
            total= total + (a[i,j]*x[j])
        #print("Linha= ",i," Total= ",total," RHS=",b[i])
        if total> b[i]:
            eviavel= False 
            break 
    return eviavel

def TrocaUnica (a,x,b,u):
    
    OMelhorObj =  RetornarValorDaFuncaoObj(x,u)
    OMelhorX = x.copy()
    #trocar valor de cada posicão do x
    xBarra = x.copy()
    objAnterior = OMelhorObj
    for j in range (col):
        xBarra[j] = 1-x[j]
        #print("J trocado = ",j,"xBarra = ",xBarra)
        #verificar se é viável o vetor retornado com a troca
        if OMelhorX[j] == xBarra[j]:
            #print("j",j,"Continue")
            continue
        else:
            if xBarra[j]==1 :
                #print("CalculoObj add","j_",j,"Obj=",objAnterior)
                objAnterior = objAnterior + u[j]
                #print("CalculoObj added","j_",j,"Obj=",objAnterior)
            else:
                #print("CalculoObj rem", "j_",j,"Obj=",objAnterior)
                objAnterior = objAnterior - u[j]      
                #print("CalculoObj rem","j_",j,"Obj=",objAnterior)
        

        #print("Uty rem-",u[j])
        #Obj = RetornarValorDaFuncaoObj(xBarra,u)
        #print("oMelhorObjTU'",OMelhorObj,"ObjTU' = ",objAnterior)
        #vericar se a função objetivo é melhor do que a que eu tinha no iníncio da TrocaUnica
        if objAnterior > OMelhorObj:
            if EViavel(a,xBarra,b):
                #print ("Obj AnteriorTU*", OMelhorObj, "Obj AtualTU*", objAnterior)
                OMelhorObj = objAnterior
                OMelhorX = xBarra.copy()
            #else:
                #print("Não é viável")
               
   
    return OMelhorObj,OMelhorX

def doisOPT(a,x,b,u):
   
    OMelhorObj =  RetornarValorDaFuncaoObj(x,u)
    OMelhorX = x.copy()
    #trocar valor de cada posicão do x
    xBarra = []
    
    for j in range (col-3):

        xBarra = x.copy()
        #print("XAntes = ",xBarra)
        xBarra[j],xBarra[j+1],xBarra[j+2],xBarra[j+3] = xBarra[j],xBarra[j+2],xBarra[j+1],xBarra[j+3]
        #print("XDepois = ",xBarra)
        #verificar se é viável o vetor retornado com a troca
        if EViavel(a,xBarra,b):
            Obj = RetornarValorDaFuncaoObj(xBarra,u)
            #print("Objetivo Viável = ",Obj)
            #vericar se a função objetivo é melhor do que a que eu tinha no iníncio da TrocaUnica
            if Obj > OMelhorObj:
                #print ("Obj Anterior", OMelhorObj, "Obj Atual", Obj)
                OMelhorObj = Obj
                OMelhorX = xBarra.copy()
                
    return OMelhorObj,OMelhorX 
   
def RetornaXPerturbado(a, x, b, alfa):
    #print("x",x, "a = ",a,"b = ",b,"alfa= ",alfa)
    xPerturbado = np.array(x).copy()
    while(True):
        for j in range(alfa):
            posicao= randint(0,col,1) 
            #print("*posicao=",posicao, "*valor = ",xPerturbado)
            xPerturbado[posicao] = 1-xPerturbado[posicao]
            #print("posicao=",posicao, "valor = ",xPerturbado[posicao])
        if (EViavel(a, xPerturbado, b)):
            break
    #print("XPerturbado Viável=",xPerturbado)        
    return xPerturbado

def calculaILS(A,RHS,row,col,Utility,x,alfa,KMAX):
    
    ObjAnterior  = 0
    XAnterior = np.zeros(col)
    print(" ObjAnterior0 = ",ObjAnterior,"XAnterior0",XAnterior)
    MelhorObj = ObjAnterior
    MelhorX = XAnterior
    print(" Melhor Obj0",MelhorObj)
    k  = 1

    while(k<=KMAX):
        print("k",k)
        XAnterior = RetornaXPerturbado(A, XAnterior, RHS, alfa)
        #ObjAnterior,XAnterior  = TrocaUnica(A,XAnterior,RHS, Utility)
        #ObjAnterior,XAnterior  = doisOPT(A,XAnterior,RHS, Utility)
        ObjAnterior,XAnterior  = doisOPTMelhor(A,XAnterior,RHS, Utility)
        print("Melhor Obj Anterior = ",MelhorObj,"Melhor X Anterior",MelhorX," Obj Encontrado = ",ObjAnterior,"X Encontrado",XAnterior)
        historicoTodos.append(ObjAnterior)
        if ObjAnterior > MelhorObj:
            print("*Melhor Obj Anterior = ",MelhorObj,"*Melhor X Anterior",MelhorX," *Melhor Obj Atual = ",ObjAnterior,"*Melhor X Atual",XAnterior)
            MelhorObj = ObjAnterior
            MelhorX = np.array(XAnterior)
            #historicoMelhor.append(MelhorObj)
        k= k+1
        
    print("*Melhor Obj Final",MelhorObj,"*Melhor X Final",MelhorX)  

def calculaVNS(A,RHS,row,col,Utility,x):
   
    ObjAnterior  = 0
    XAnterior = x.copy()
    XAnterior = HeuristicaGulosa(A, RHS, Utility, XAnterior)
    ObjAnterior=RetornarValorDaFuncaoObj(x,Utility)
    #print("--ObjAnterior0 = ",ObjAnterior,"--XAnterior0",XAnterior)
    MelhorObj = ObjAnterior
    MelhorX = XAnterior
    #print("--Melhor Obj0",MelhorObj)
    
    naoHouvemelhoria = True
    while(naoHouvemelhoria):
        
        ObjAnterior,XAnterior  = doisOPTMelhor(A,XAnterior,RHS, Utility)
        #ObjAnterior,XAnterior  = doisOPT(A,XAnterior,RHS, Utility)    
        #print("Melhor Obj Anterior = ",MelhorObj,"Melhor X Anterior",MelhorX," Obj Encontrado = ",ObjAnterior,"X Encontrado",XAnterior)
        #historicoTodos.append(ObjAnterior)
        if ObjAnterior > MelhorObj:
            
            #diferenca = (ObjAnterior - MelhorObj)/ObjAnterior
            
            #print("*Melhor Obj Anterior = ",MelhorObj,"*Melhor X Anterior",MelhorX," *Melhor Obj Atual = ",ObjAnterior,"*Melhor X Atual",XAnterior)
            MelhorObj = ObjAnterior
            MelhorX = XAnterior.copy()
            #historicoMelhor.append(MelhorObj)
            #if diferenca <= 0.05:
            #        naoHouvemelhoria = False
                    
        else:
            ObjAnterior,XAnterior  = TrocaUnica(A,XAnterior,RHS, Utility)
            if ObjAnterior > MelhorObj:
                
                #print("**Melhor Obj Anterior = ",MelhorObj,"**Melhor X Anterior",MelhorX," **Melhor Obj Atual = ",ObjAnterior,"**Melhor X Atual",XAnterior)
                MelhorObj = ObjAnterior
                MelhorX = XAnterior.copy()
                #historicoMelhor.append(MelhorObj)  
                
            else:
                naoHouvemelhoria = False        
            
            
    print("*VNS - *Melhor Obj Final",MelhorObj,"*Melhor X Final",MelhorX)  
    print("fo certa=",RetornarValorDaFuncaoObj(MelhorX, Utility))
    
def doisOPTMelhor(a,x,b,u):    
    
    OMelhorObj =  RetornarValorDaFuncaoObj(x,u)
    OMelhorX = x.copy()
    #trocar valor de cada posicão do x
    xBarra = []
    firstimprove = False
    for i in range (col):
        #if firstimprove == True:
        #    break
        j = i+1    
        while j < col:
            
            xBarra = x.copy().tolist()
            #print("XAntes = ",xBarra,"i = ",i,"j = ",j)
            #xBarra = Inverte(i,j,x)
            
            before = xBarra[:i]
            middle= xBarra[(i):(j)]
            #print (middle)
            middle.reverse()
            after=xBarra[(j):]
            #print (after)
            
            xBarra = before + middle + after
            xBarra = list(map(int, xBarra))
            xBarra = np.array(xBarra)
            
            Obj = OMelhorObj
            h=i
            #print("OMelhorX2OPT = ",OMelhorX)
            #print("XDepois2opt = ",xBarra)
            while h < (j):
                if OMelhorX[h] ==  xBarra[h]:
                    h=h+1  
                    #print("CalculoObj continue")
                    continue
                    
                else:
                   
                    if xBarra[h]==1:
                        #print("CalculoObj add", u[h],"h_",h,"j_",j,"Obj=",Obj)
                        Obj = Obj + u[h]
                        #print("CalculoObj added", u[h],"h_",h,"j_",j,"Obj=",Obj)
                    else:
                        #print("CalculoObj rem", u[h],"h_",h,"j_",j,"Obj=",Obj)
                        Obj = Obj - u[h]
                        #print("CalculoObj remed", u[h],"h_",h,"j_",j,"Obj=",Obj)
                h=h+1                
                
            
            
            #verificar se é viável o vetor retornado com a troca
            #Obj = RetornarValorDaFuncaoObj(xBarra,u)
            
            #print("omelhorobj2OPT",OMelhorObj,"Obj2OPT' = ",Obj)
            #vericar se a função objetivo é melhor do que a que eu tinha no iníncio da TrocaUnica
            if Obj > OMelhorObj:
                if EViavel(a,xBarra,b):
                    print ("Obj Anterior", OMelhorObj, "Obj Atual", Obj)
                    OMelhorObj = Obj
                    OMelhorX = xBarra.copy()
                    #firstimprove = True
                    #break
            j = j+1            
            
    return OMelhorObj,OMelhorX


def calculaLS(A,RHS,row,col,Utility,x):
  
    XAnterior = HeuristicaGulosa(A, RHS, Utility, x)
    ObjAnterior=RetornarValorDaFuncaoObj(x,Utility)
    #print(" ObjAnterior0 = ",ObjAnterior,"XAnterior0",XAnterior)
    MelhorObj = ObjAnterior
    MelhorX = XAnterior
    #print(" Melhor Obj0",MelhorObj)
    
    naoHouvemelhoria = True
    while(naoHouvemelhoria):
        
        ObjAnterior,XAnterior  = doisOPTMelhor(A,XAnterior,RHS, Utility)
        
        #ObjAnterior,XAnterior  = doisOPT(A,XAnterior,RHS, Utility)    
        #print("Melhor Obj Anterior = ",MelhorObj,"Melhor X Anterior",MelhorX," Obj Encontrado = ",ObjAnterior,"X Encontrado",XAnterior)
        #historicoTodos.append(ObjAnterior)
        if ObjAnterior > MelhorObj:
            
            #diferenca = (ObjAnterior - MelhorObj)/ObjAnterior
            
            #print("*Melhor Obj Anterior = ",MelhorObj,"*Melhor X Anterior",MelhorX," *Melhor Obj Atual = ",ObjAnterior,"*Melhor X Atual",XAnterior)
            MelhorObj = ObjAnterior
            MelhorX = XAnterior.copy()
            #historicoMelhor.append(MelhorObj)
            #if diferenca <= 0.05:
            #    naoHouvemelhoria = False    
        else:
        
            naoHouvemelhoria = False        
        
    print("*LS2OPT *Melhor Obj Final",MelhorObj,"*Melhor X Final",MelhorX)  
    print("fo certa=",RetornarValorDaFuncaoObj(MelhorX, Utility))
    
def HeuristicaGulosa(a,b,c,x):
    #print("gulosa imput",x)
    cc = c.copy()
    xx = x.copy() 
    Viavel = True
    maxIndex  = 0
    while Viavel:
        maxIndex = np.argmax(cc)
        #print("cc",cc)
        #print("maxIndex",maxIndex)
        xx[maxIndex] = 1
        cc[maxIndex] = 0 
        Viavel = EViavel(a,xx,b)
    xx[maxIndex] = 0
    #print("gulosa x ",xx)
    return xx

In [6]:
A = []
row = 0
col = 0
RHS = []
Utility  = []
x =[]

C5I10 = ""
C5I10 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C5I10.dat"
A,RHS,row,col,Utility = leArquivo(C5I10)
x = np.zeros(col) 
#calculaILS(A,RHS,row,col,Utility,x,2,1)
calculaVNS(A,RHS,row,col,Utility,x)
calculaLS(A,RHS,row,col,Utility,x)

C30I20 = ""
C30I20 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C30I20.dat"
A,RHS,row,col,Utility = leArquivo(C30I20)
x = np.zeros(col) 
#calculaILS(A,RHS,row,col,Utility,x,2,40)
calculaVNS(A,RHS,row,col,Utility,x)
calculaLS(A,RHS,row,col,Utility,x)

C150I200 = ""
C150I200 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C150I200.dat"
A,RHS,row,col,Utility = leArquivo(C150I200)
x = np.zeros(col) 
#calculaILS(A,RHS,row,col,Utility,x,6,50)
calculaVNS(A,RHS,row,col,Utility,x)
calculaLS(A,RHS,row,col,Utility,x)

C500I500 =""
C500I500 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C500I500.dat"
A,RHS,row,col,Utility = leArquivo(C500I500)
x = np.zeros(col) 
#calcula(A,RHS,row,col,Utility,x,10,20)
calculaVNS(A,RHS,row,col,Utility,x)
calculaLS(A,RHS,row,col,Utility,x)

C500I700 = ""
C500I700 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C500I700.dat"
A,RHS,row,col,Utility = leArquivo(C500I700)
x = np.zeros(col)
#calcula(A,RHS,row,col,Utility,x,460,6)
calculaVNS(A,RHS,row,col,Utility,x)
calculaLS(A,RHS,row,col,Utility,x)

C700I500 = ""
C700I500 = r"C:\Users\User\Documents\Mestrado\Combinatorial Otimization\Trabalho2\Dados\Knap_C700I500.dat"
A,RHS,row,col,Utility = leArquivo(C700I500)
x = np.zeros(col)
#calcula(A,RHS,row,col,Utility,x,240,6)
calculaVNS(A,RHS,row,col,Utility,x)
calculaLS(A,RHS,row,col,Utility,x)


*VNS - *Melhor Obj Final 3.84807 *Melhor X Final [1. 1. 1. 1. 1. 0. 0. 1. 1. 0.]
fo certa= 3.84807
*LS2OPT *Melhor Obj Final 3.84807 *Melhor X Final [1. 1. 1. 1. 1. 0. 0. 1. 1. 0.]
fo certa= 3.84807
*VNS - *Melhor Obj Final 9.257930000000002 *Melhor X Final [0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1.]
fo certa= 9.257930000000002
*LS2OPT *Melhor Obj Final 9.257930000000002 *Melhor X Final [0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1.]
fo certa= 9.257930000000002
*VNS - *Melhor Obj Final 89.60858999999998 *Melhor X Final [1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0.
 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1.
 0. 1

*VNS - *Melhor Obj Final 210.5125100000001 *Melhor X Final [1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0.
 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0.
 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0.
 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0.
 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0

In [7]:
#plt.plot(historicoMelhor, "-o")
#plt.plot(historicoTodos, "-o")
#plt.ylabel('Objetive Function')
#plt.xlabel('# Round')

In [8]:
x = np.array([0,1,2,3,4,5,6,7,8,9])
xBarra = []
for i in range (10):
    j = i+1    
    while j < 10:

        xBarra = x.copy().tolist()
        #print("XAntes = ",xBarra,"i = ",i,"j = ",j)
        #xBarra = Inverte(i,j,x)

        before = xBarra[:i]
        middle= xBarra[(i):(j)]
        #print (middle)
        middle.reverse()
        after=xBarra[(j):]
        #print (after)

        xBarra = before + middle + after
        xBarra = list(map(int, xBarra))
        xBarra = np.array(xBarra)
        print(xBarra)
        j=j+1
        h=i
        while h < (j):
            print ("xBarra[h]=",h)
            h=h+1      

[0 1 2 3 4 5 6 7 8 9]
xBarra[h]= 0
xBarra[h]= 1
[1 0 2 3 4 5 6 7 8 9]
xBarra[h]= 0
xBarra[h]= 1
xBarra[h]= 2
[2 1 0 3 4 5 6 7 8 9]
xBarra[h]= 0
xBarra[h]= 1
xBarra[h]= 2
xBarra[h]= 3
[3 2 1 0 4 5 6 7 8 9]
xBarra[h]= 0
xBarra[h]= 1
xBarra[h]= 2
xBarra[h]= 3
xBarra[h]= 4
[4 3 2 1 0 5 6 7 8 9]
xBarra[h]= 0
xBarra[h]= 1
xBarra[h]= 2
xBarra[h]= 3
xBarra[h]= 4
xBarra[h]= 5
[5 4 3 2 1 0 6 7 8 9]
xBarra[h]= 0
xBarra[h]= 1
xBarra[h]= 2
xBarra[h]= 3
xBarra[h]= 4
xBarra[h]= 5
xBarra[h]= 6
[6 5 4 3 2 1 0 7 8 9]
xBarra[h]= 0
xBarra[h]= 1
xBarra[h]= 2
xBarra[h]= 3
xBarra[h]= 4
xBarra[h]= 5
xBarra[h]= 6
xBarra[h]= 7
[7 6 5 4 3 2 1 0 8 9]
xBarra[h]= 0
xBarra[h]= 1
xBarra[h]= 2
xBarra[h]= 3
xBarra[h]= 4
xBarra[h]= 5
xBarra[h]= 6
xBarra[h]= 7
xBarra[h]= 8
[8 7 6 5 4 3 2 1 0 9]
xBarra[h]= 0
xBarra[h]= 1
xBarra[h]= 2
xBarra[h]= 3
xBarra[h]= 4
xBarra[h]= 5
xBarra[h]= 6
xBarra[h]= 7
xBarra[h]= 8
xBarra[h]= 9
[0 1 2 3 4 5 6 7 8 9]
xBarra[h]= 1
xBarra[h]= 2
[0 2 1 3 4 5 6 7 8 9]
xBarra[h]= 1
xBarra[h]= 2
xBar

In [9]:
x = np.zeros(10)
for j in range (10):
    print("J trocado = ",j,"xBarra = ",x,"valor antes = ", x[j])        
    x[j] = 1-x[j]
    print("J trocado = ",j,"xBarra = ",x,"valor depois = ", x[j])        
            

J trocado =  0 xBarra =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] valor antes =  0.0
J trocado =  0 xBarra =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] valor depois =  1.0
J trocado =  1 xBarra =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] valor antes =  0.0
J trocado =  1 xBarra =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.] valor depois =  1.0
J trocado =  2 xBarra =  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.] valor antes =  0.0
J trocado =  2 xBarra =  [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] valor depois =  1.0
J trocado =  3 xBarra =  [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.] valor antes =  0.0
J trocado =  3 xBarra =  [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.] valor depois =  1.0
J trocado =  4 xBarra =  [1. 1. 1. 1. 0. 0. 0. 0. 0. 0.] valor antes =  0.0
J trocado =  4 xBarra =  [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.] valor depois =  1.0
J trocado =  5 xBarra =  [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.] valor antes =  0.0
J trocado =  5 xBarra =  [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.] valor depois =  1.0
J trocado =  6 xBarra =  [1. 1. 1. 1. 1. 1. 0. 0. 0. 0.] valor antes =  0.0
J troc